# Urban growth modeling in GRASS GIS: Parallel computing case study
The purpose of this notebook is to demonstrate several parallel computing principles and how they are implemented in GRASS GIS.
We use FUTURES urban growth model implemented as a GRASS GIS addon [r.futures](https://grass.osgeo.org/grass-stable/manuals/addons/r.futures.html).

This notebook requires prepared dataset [available here](https://doi.org/10.5281/zenodo.6577922). This dataset is a GRASS GIS Location containing:
 * [NLCD 2001-2019](https://www.mrlc.gov/) (land use and impervious surface descriptor)
 * [US county boundaries](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html)
 * [US-PAD protected areas](https://www.usgs.gov/programs/gap-analysis-project/science/pad-us-data-overview)
 * [USGS DEM](https://www.usgs.gov/3d-elevation-program/about-3dep-products-services)
 
Population files are [available for download here](https://doi.org/10.5281/zenodo.6577903).

The required software includes
 * _GRASS GIS v8.2_ with the following addons: _r.futures, r.mapcalc.tiled, r.sample.category_
 * _R_ with the following packages: _lme4, optparse, MuMIn, snow_
 * _GNU Parallel_
 * _Python 3_ with packages _pandas_
 
See [documentation](https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html) how to use GRASS GIS in a Jupyter Notebook. This notebook was written for Ubuntu OS, see [instructions](https://github.com/chaedri/GIS714-assignments) how to run the notebook on Windows. On Windows, use for example WSL to run Bash code.

The expected computing resources for the notebook as intended are 12 cores and 32GB memory.

In [ ]:
nprocs = 12

This notebook combines Python 3 and Bash cells. By default a code cell is in Python. We use IPython [cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cell-magics) including `%%bash`, `%%time`, and `%%writefile`.

## Setting up
Create a directory on your disk with enough disk space (100GB), unzip FUTURES_SE_USA.zip and population_SE_counties_2001-2100_SSP2.zip:
```
├── FUTURES_SE_USA
│   ├── PERMANENT
│   └── tutorial
├── observed_population_SE_counties_2001-2019.csv
└── projected_population_SE_counties_2020-2100_SSP2.csv

```

Change the current directory to wherever you unzipped the data, example below (modify as needed):

In [ ]:
import os

os.chdir("~/FUTURES_case_study/")

Import Python packages and initialize GRASS GIS session. On Windows, see [instructions here](https://github.com/chaedri/GIS714-assignments).

In [ ]:
import subprocess
import sys
import pathlib
import json
import pandas as pd
from IPython.display import Image

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init(".", "FUTURES_SE_USA", "tutorial")

Install GRASS addons:

In [ ]:
gs.run_command("g.extension", extension="r.futures")
gs.run_command("g.extension", extension="r.mapcalc.tiled")
gs.run_command("g.extension", extension="r.sample.category")

## Data preprocessing

List dataset layers:

In [ ]:
%%bash
g.list type=raster,vector -p

### Process county boundaries
Extract counties in Southeast US states (Alabama, Florida, Georgia, North Carolina, South Carolina, Tennessee)
and convert GEOID columns to integer.

In [ ]:
%%bash
v.extract tl_2021_us_county output=SE_counties where="STATEFP in ('01', '12', '13', '37', '45', '47')" --q
v.db.addcolumn SE_counties column="state integer" --q
v.db.addcolumn SE_counties column="county integer" --q
v.db.update SE_counties col=state qcol="CAST(STATEFP AS integer)" --q
v.db.update SE_counties col=county qcol="CAST(GEOID AS integer)" --q

In [ ]:
m = gj.Map()
m.d_vect(map="SE_counties")
m.show()

Split and rasterize states for further parallelization steps:

In [ ]:
states = [1, 12, 13, 37, 45, 47]
gs.use_temp_region()
for state in states:
    gs.run_command(
        "v.extract",
        input="SE_counties",
        where=f"state == '{state}'",
        output=f"state_{state}",
    )
    gs.run_command("g.region", vector=f"state_{state}", align="nlcd_2019")
    gs.run_command(
        "v.to.rast",
        input=f"state_{state}",
        output=f"state_{state}",
        use="attr",
        attribute_column="county",
    )
gs.del_temp_region()

### DEM to slope
Compute slope with [r.slope.aspect](https://grass.osgeo.org/grass-stable/manuals/r.slope.aspect.html) which uses OpenMP for parallelization:

In [ ]:
gs.run_command(
    "r.slope.aspect", elevation="DEM", slope="slope", flags="e", nprocs=nprocs
)

In [ ]:
m = gj.Map()
m.d_rast(map="slope")
m.show()

### Protected land
Rasterize protected areas to later include them in a mask. We use GridModule to split the computation in tiles:

In [ ]:
%%python
from grass.pygrass.modules.grid import GridModule

grid = GridModule(
    "v.to.rast",
    input="protected",
    output="protected",
    type="area",
    use="cat",
    processes=12,
    patch_backend="r.patch",
    quiet=True,
)
grid.run()

In [ ]:
m = gj.Map()
m.d_rast(map="protected")
m.show()

### Process NLCD data
Most of our predictors we will derive from NLCD data (land cover type and impervious descriptor products). With r.reclass we create water, wetland, forest, roads, urban rasters.
Note that those rasters are virtual (they behave the same way, but are only pointing to the original NLCD raster),
so reclassification is very fast.

In [ ]:
NLCD_years = [2001, 2004, 2006, 2008, 2011, 2013, 2016, 2019]
NLCD_start_end_years = [2001, 2019]
# water (1 or no data)
gs.write_command(
    "r.reclass", input="nlcd_2019", output="water", rules="-", stdin="11 = 1"
)
# binary wetlands
gs.write_command(
    "r.reclass",
    input="nlcd_2019",
    output="wetlands",
    rules="-",
    stdin="90 95 = 1 \n * = 0",
)
for year in NLCD_years:
    gs.write_command(
        "r.reclass",
        input=f"nlcd_{year}",
        output=f"urban_{year}",
        rules="-",
        stdin="21 22 23 24 = 1\n* = 0",
    )
for year in NLCD_start_end_years:
    gs.write_command(
        "r.reclass",
        input=f"nlcd_{year}",
        output=f"forest_{year}",
        rules="-",
        stdin="41 42 43 = 1",
    )
    gs.write_command(
        "r.reclass",
        input=f"nlcd_descriptor_{year}",
        output=f"roads_{year}",
        rules="-",
        stdin="20 21 22 23 = 1",
    )
    gs.write_command(
        "r.reclass",
        input=f"nlcd_descriptor_{year}",
        output=f"urban_no_roads_{year}",
        rules="-",
        stdin="24 25 26 = 1\n* = 0",
    )

In Bash use background processing (append &) to compute distance to water, forest, and roads in parallel since these are independent computations. Command `wait` forces to wait for the background processes to finish.
Once the distance is computed, we use raster algebra to transform it logarithmically.

In [ ]:
%%bash
r.grow.distance input=water distance=dist_to_water -m --q &
r.grow.distance input=forest_2001 distance=dist_to_forest_2001 -m --q &
r.grow.distance input=forest_2019 distance=dist_to_forest_2019 -m --q &
r.grow.distance input=roads_2001 distance=dist_to_roads_2001 -m --q &
r.grow.distance input=roads_2019 distance=dist_to_roads_2019 -m --q &
wait
r.mapcalc "log_dist_to_water = log(dist_to_water + 1)" --q &
r.mapcalc "log_dist_to_forest_2001 = log(dist_to_forest_2001 + 1)" --q &
r.mapcalc "log_dist_to_forest_2019 = log(dist_to_forest_2019 + 1)" --q &
r.mapcalc "log_dist_to_roads_2001 = log(dist_to_roads_2001 + 1)" --q &
r.mapcalc "log_dist_to_roads_2019 = log(dist_to_roads_2019 + 1)" --q &
wait

In [ ]:
m = gj.Map()
m.d_rast(map="log_dist_to_forest_2019")
m.show()

As another predictor, we compute wetland density (percentage of wetland in 1 km squared circular neighborhood). Module [r.neighbors](https://grass.osgeo.org/grass-stable/manuals/r.neighbors.html) is internally parallelized using OpenMP, so we can use `nprocs` option:

In [ ]:
gs.run_command(
    "r.neighbors",
    input="wetlands",
    output="wetland_density",
    size=37,
    method="average",
    flags="c",
    nprocs=nprocs,
)

FUTURES uses a special predictor called development pressure, which can be computed with [r.futures.devpressure](https://grass.osgeo.org/grass-stable/manuals/addons/r.futures.devpressure.html), which is internally parallelized.
Since we need to compute it for 2 years, we use a hybrid approach which runs both command as background process and each of them runs in parallel.
To do that we split the number of available processes so that each r.futures.devpressure process gets half of the available processes:

In [ ]:
%%bash -s "{int(nprocs/2)}"
r.futures.devpressure input=urban_no_roads_2001 output=devpressure_2001 size=30 gamma=0.5 nprocs=$1 scaling_factor=0.1 &
r.futures.devpressure input=urban_no_roads_2019 output=devpressure_2019 size=30 gamma=0.5 nprocs=$1 scaling_factor=0.1 &
wait

In [ ]:
m = gj.Map()
m.d_rast(map="devpressure_2001")
m.show()

### Mask
Compute mask to avoid urban growth simulation in water, protected areas, and outside the study area. We use [r.mapcalc.tiled](https://grass.osgeo.org/grass-stable/manuals/addons/r.mapcalc.tiled.html):

In [ ]:
gs.run_command(
    "r.mapcalc.tiled",
    expression="masking = if((isnull(protected) &&  isnull(water) && nlcd_2019), 1, null())",
    nprocs=nprocs,
)

In [ ]:
m = gj.Map()
m.d_rast(map="masking")
m.show()

## Demand computation
Here we compute how much land will be developed in each step of the simulation.
Logarithmic curves are fit to per-capita land consumption data derived from NLCD time series and observed population for each county.

This computation is parallelized by state. First we create a Python script that takes the state as an input parameter,
sets the computational region to the state extent, excludes roads from the computation, and runs [r.futures.demand](https://grass.osgeo.org/grass-stable/manuals/addons/r.futures.demand.html),
creating an output CSV and a plot specific to that state.

In [ ]:
%%writefile demand_for_state.py
import sys
import grass.script as gs

# input parameter: state FIPS code
state = sys.argv[1]

gs.run_command("g.mapsets", mapset="tutorial", operation="add")
gs.run_command("g.region", raster=f"state_{state}")
gs.mapcalc("MASK = if (isnull(roads), 1, null())")
gs.run_command(
    "r.futures.demand",
    subregions=f"state_{state}",
    development=[
        f"urban_{year}" for year in [2001, 2004, 2006, 2008, 2011, 2013, 2016, 2019]
    ],
    observed_population="observed_population_SE_counties_2001-2019.csv",
    projected_population="projected_population_SE_counties_2020-2100_SSP2.csv",
    simulation_times=list(range(2019, 2051)),
    method="logarithmic",
    demand=f"demand_{state}.csv",
    plot=f"demand_{state}.png",
    overwrite=True,
)

For each state generate grass command calling the script within a temporary mapset and append the line to a file `demand_jobs.sh`.
Run these commands in parallel with GNU Parallel.

In [ ]:
%%bash -s "{nprocs}"
rm -f demand_jobs.sh
for S in 1 12 13 37 45 47
do
    echo grass --tmp-mapset FUTURES_SE_USA --exec python demand_for_state.py ${S} >> demand_jobs.sh
done
cat demand_jobs.sh
parallel -j ${1} < demand_jobs.sh 2> log.txt

Visualize results for one state:

In [ ]:
Image("demand_45.png")

## Development Potential
### Predictor Sampling
First we need a layer representing newly developed areas between 2001 and 2019 (value 1) and areas that didn't transition (value 0):

In [ ]:
gs.run_command(
    "r.mapcalc.tiled",
    expression="urban_change = if(urban_2001 == 0, if(urban_2019 == 1, 1, 0), null())",
    nprocs=nprocs,
)

Similarly to Demand computation, we create a script that is then executed with different states.
Since we need to patch the results together, we won't use temporary mapset.
Setting the mask ensures area outside of it won't be sampled.

In [ ]:
%%writefile sampling_for_state.py
import sys
import grass.script as gs

state = sys.argv[1]

gs.run_command("g.mapsets", mapset="tutorial", operation="add")
gs.run_command("g.region", raster=f"state_{state}")
gs.run_command("r.mask", raster=f"state_{state}")
# create an identical, virtual map of counties with different name to later simplify patching of attribute tables
gs.write_command(
    "r.reclass", input=f"state_{state}", output="counties", rules="-", stdin="* = *"
)
gs.run_command(
    "r.sample.category",
    input="urban_change",
    output=f"sample_{state}",
    sampled=[
        "counties",
        "devpressure_2001",
        "log_dist_to_forest_2001",
        "log_dist_to_roads_2001",
        "log_dist_to_water",
        "slope",
        "wetland_density",
    ],
    npoints=[10000, 5000],
    random_seed=1,
)
gs.run_command("r.mask", flags="r")

In [ ]:
%%bash -s "{nprocs}"
rm -f sampling_jobs.sh
for S in 1 12 13 37 45 47
do
    # remove mapset to start fresh after previous run
    rm -rf FUTURES_SE_USA/sampling_${S}
    echo grass -c FUTURES_SE_USA/sampling_${S} --exec python sampling_for_state.py ${S} >> sampling_jobs.sh
done
parallel -j ${1} < sampling_jobs.sh 2> log.txt

Here we patch the results:

In [ ]:
gs.run_command(
    "v.patch",
    input=[f"sample_{state}@sampling_{state}" for state in states],
    output="samples",
    flags="e",
)

Zoom in to see samples in newly developed and undeveloped areas:

In [ ]:
gs.run_command("g.region", n=1496445, s=1473765, e=1373865, w=1346175, save="zoomin")
m = gj.Map(saved_region="zoomin")
m.d_rast(map="urban_change")
m.d_vect(map="samples", size=10, fill_color="red", icon="basic/pin")
m.show()

### Potential
Runs r.futures.potential to select the "best" model and compute regression coefficients.
The best model selection runs in parallel (parallelized in R).

In [ ]:
gs.run_command(
    "r.futures.potential",
    input="samples",
    output="best_model.csv",
    columns=[
        "devpressure_2001",
        "log_dist_to_forest_2001",
        "log_dist_to_roads_2001",
        "log_dist_to_water",
        "slope",
        "wetland_density",
    ],
    developed_column="urban_change",
    subregions_column="counties",
    random_column="devpressure_2001",
    min_variables=3,
    nprocs=nprocs,
    flags="d",
    dredge_output="all_models.csv",
)

In [ ]:
pd.read_csv("all_models.csv", index_col=0)

In [ ]:
pd.read_csv("best_model.csv", index_col=0)

Rename the distance column headers to use 2019 instead for projections:

In [ ]:
%%bash
sed -i 's/_2001/_2019/g' best_model.csv

### Calibration
Here we derive the distribution of the historical patch sizes per county. [r.futures.calib](https://grass.osgeo.org/grass-stable/manuals/addons/r.futures.calib.html) is internally parallelized,
so depending on the number of available cores, you can select 1 or more cores for each r.futures.calib run.

In [ ]:
%%writefile calibration_for_state.py
import sys
import grass.script as gs

state = sys.argv[1]

gs.run_command("g.mapsets", mapset="tutorial", operation="add")
gs.run_command("g.region", raster=f"state_{state}")
gs.run_command(
    "r.futures.calib",
    flags="sl",
    development_start="urban_2001",
    development_end="urban_2019",
    subregions=f"state_{state}",
    patch_threshold=1800,
    patch_sizes=f"patch_sizes_{state}.csv",
    nprocs=2,
)

In [ ]:
%%bash -s "{int(nprocs/2)}"
rm -f calibration_jobs.sh
for S in 1 12 13 37 45 47
do
    echo grass --tmp-mapset FUTURES_SE_USA/ --exec python calibration_for_state.py ${S} >> calibration_jobs.sh
done
parallel -j $1 < calibration_jobs.sh 2> log.txt

### Patch Growing Algorithm
Putting all the intermediate results together, we can now finally run the simulation from 2019 until 2050.

We will run 3 simulations in parallel. r.futures.pga
will be executed many times (number of stochastic runs * number of states = 10 * 6 = 60).
If more memory is available, more cores can be used.

In [ ]:
%%writefile simulation_for_state.py
import sys
import grass.script as gs

state, seed = sys.argv[1:3]

gs.run_command("g.mapsets", mapset="tutorial", operation="add")
gs.run_command("g.region", raster=f"state_{state}")
gs.run_command("r.mask", raster="masking")
gs.run_command(
    "r.futures.pga",
    developed="urban_2019",
    development_pressure="devpressure_2019",
    compactness_mean=0.4,
    compactness_range=0.1,
    discount_factor=0.5,
    predictors=[
        "log_dist_to_forest_2019",
        "log_dist_to_roads_2019",
        "log_dist_to_water",
        "slope",
        "wetland_density",
    ],
    n_dev_neighbourhood=30,
    devpot_params="best_model.csv",
    num_neighbors=4,
    seed_search="probability",
    development_pressure_approach="gravity",
    gamma=0.5,
    scaling_factor=0.1,
    subregions=f"state_{state}",
    demand=f"demand_{state}.csv",
    num_steps=31,
    output=f"out_state_{state}_seed_{seed}",
    patch_sizes=f"patch_sizes_{state}.csv",
    incentive_power=2,
    random_seed=seed,
)
gs.run_command("r.mask", flags="r")


In [ ]:
%%bash
rm -f pga_jobs.sh
for SEED in {1..10}
do
    for STATE in 1 12 13 37 45 47
    do
        rm -rf FUTURES_SE_USA/pga_${STATE}_${SEED}
        echo grass -c FUTURES_SE_USA/pga_${STATE}_${SEED} --exec python simulation_for_state.py ${STATE} ${SEED} >> pga_jobs.sh
    done
done
parallel -j 6 < pga_jobs.sh 2> log.txt

Afterwards, we patch the results together. Tool [r.patch](https://grass.osgeo.org/grass-stable/manuals/r.patch.html) is internally parallelized, so we can use that extra speed up if we have available cores.

In [ ]:
%%bash -s "{int(nprocs/2)}"
rm -f patch_jobs.sh
for SEED in {1..10}
do
    MAPS=$(grass --tmp-mapset FUTURES_SE_USA/ --exec \
           g.list type=raster pattern="out_state_*_seed_${SEED}" mapset="*" -m separator=comma)
    rm -rf FUTURES_SE_USA/results_${SEED}
    echo grass -c FUTURES_SE_USA/results_${SEED} --exec r.patch input=${MAPS} output="out_seed_${SEED}" nprocs=2 >> patch_jobs.sh
done
parallel -j ${1} < patch_jobs.sh 2> log.txt

In [ ]:
gs.run_command(
    "r.colors",
    map="out_seed_1@results_1",
    raster="out_state_12_seed_1@pga_12_1",
)
m = gj.Map()
m.d_rast(map="out_seed_1@results_1")
m.show()

### Future Development Probability
By aggregating the stochastic runs we can compute the projected development probability. First we reclassify output
to binary developed/undeveloped results. Then we run [r.series](https://grass.osgeo.org/grass-stable/manuals/r.series.html) in parallel to compute how many times a cell was developed
and then divide that by number of runs.

In [ ]:
%%writefile reclass.txt
-1 0 = 0
1 thru 100 = 1
* = 0


In [ ]:
for seed in range(1, 11):
    gs.run_command(
        "r.reclass",
        input=f"out_seed_{seed}@results_{seed}",
        output=f"out_seed_{seed}",
        rules="reclass.txt",
    )
gs.run_command(
    "r.series",
    input=[f"out_seed_{seed}" for seed in range(1, 11)],
    output="probability",
    method="sum",
    weights=[0.1] * 10,
    nprocs=nprocs,
)

In [ ]:
gs.run_command(
    "g.region",
    n=1439421,
    s=1414322,
    w=1335588,
    e=1366210,
    align="probability",
    save="zoomin2",
)
m = gj.Map(saved_region="zoomin2")
m.d_rast(map="probability")
m.d_rast(map="urban_2019", values=1)
m.show()

## Forest loss analysis
Here we compute future forest loss due to development, demonstrating how to parallelize computations using Python multiprocessing.
The goal is to compute forest loss for each of 30x30 km tiles in parallel across the landscape to capture large scale forest loss patterns.
To do that we use `GRASS_REGION` environmental variable and set different region for each seed and tile combination.

We will run this as a Python script (not directly from the cell) because Python multiprocessing sometimes doesn't work well within a Jupyter Notebook.
Results that include tile coordinate, seed, and number of forest cells are written as JSON to a file.

In [ ]:
%%writefile forest_loss.py
import os
import sys
import json
from math import ceil
from multiprocessing import Pool

import grass.script as gs
from grass.exceptions import CalledModuleError


def forest_loss_stats(params):
    """Compute projected forest loss for selected area.
    This function can be safely run in parallel."""
    seed, region = params
    # create unique temporary map names
    forest_loss = f"forest_{seed}_{region['n']}_{region['e']}"
    # copy and modify environment to change region based on input
    env = os.environ.copy()
    env["GRASS_REGION"] = gs.region_env(align="forest_2019", **region)
    # pass the environment, so that the computations run with different region than the current one
    # compute lost forest comparing to 2019
    gs.mapcalc(
        f"{forest_loss} = if (forest_2019 && out_seed_{seed}@results_{seed} >= 0, 1, 0)",
        env=env,
        quiet=True,
    )
    # get number of cells for 0 (no change) and 1 (forest lost) category
    results = dict(
        gs.parse_command(
            "r.stats",
            input=forest_loss,
            flags="cn",
            parse=(gs.parse_key_val, {"sep": " ", "val_type": int}),
            env=env,
            quiet=True,
        )
    )
    # add N, E as a center of the region, and add seed to results
    results["n"] = (region["n"] + region["s"]) / 2
    results["e"] = (region["e"] + region["w"]) / 2
    results["seed"] = seed
    # remove temporary maps
    gs.run_command("g.remove", type="raster", name=forest_loss, flags="f", quiet=True)
    # return dictionary with results
    return results


if __name__ == "__main__":
    nproc = int(sys.argv[1])
    current = gs.region()
    regions = []
    tile = 1000
    # create a region where each cell is a 30x30 km tile
    gs.run_command("g.region", res=tile * current["nsres"], flags="a", save="tiles")
    env = os.environ.copy()
    env["GRASS_REGION"] = gs.region_env(region="tiles")
    grid_region = gs.region(env=env)
    # save extents of each tile
    for row in range(int(grid_region["rows"])):
        for col in range(int(grid_region["cols"])):
            s = float(grid_region["s"]) + row * float(grid_region["nsres"])
            n = float(grid_region["s"]) + (row + 1) * float(grid_region["nsres"])
            w = float(grid_region["w"]) + col * float(grid_region["ewres"])
            e = float(grid_region["w"]) + (col + 1) * float(grid_region["ewres"])
            regions.append(
                {
                    "n": n,
                    "s": s,
                    "w": w,
                    "e": e,
                }
            )

    params = []
    # collect parameters (for each seed and tile)
    for seed in range(1, 11):
        for region in regions:
            params.append((seed, region))

    # execute forest loss computation for each tile in parallel
    with Pool(processes=nproc) as pool:
        results = pool.map_async(forest_loss_stats, params).get()
        # write results as json to file
        with open("forest_results.json", "w") as f:
            json.dump(results, f)


Execute the file with number of processes as an input:

In [ ]:
%run forest_loss.py {nprocs}

Load results, process them with Pandas and write a raster:

In [ ]:
with open("forest_results.json", "r") as f:
    results = json.load(f)

df = pd.DataFrame(results)
df = df[df["1"].notna()].groupby(["n", "e"])[["1"]].mean()
df["km"] = df["1"] * 900 / 1000000

csv = df.drop(columns=["1"]).to_csv(index=True, header=False)
env = os.environ.copy()
env["GRASS_REGION"] = gs.region_env(region="tiles")
gs.write_command(
    "r.in.xyz",
    stdin=csv,
    input="-",
    x=2,
    y=1,
    output="forest_loss",
    method="mean",
    separator="comma",
    env=env,
)
gs.run_command("r.colors", map="forest_loss", color="forest_cover")

Visualize:

In [ ]:
m = gj.Map()
m.d_rast(map="forest_loss")
m.d_vect(map="SE_counties", fill_color="none", color="grey")
m.d_legend(raster="forest_loss", title="Loss of forest [km sq]")
m.show()